#### Task

The task is to find out the total number of viewer across all shows for the channel **BAT**.

#### Data Description

- The gennum (ABC) files contains show names and number of viewers.
- The genchan files contains show names and channel.

One trick is to read all of them into Spark with a pattern matching `?` symbol at the end.

In [1]:
# print out the dataset
show_views_file = sc.textFile("join2_gennum?.txt")
show_channel_file = sc.textFile("join2_genchan?.txt")
print show_views_file.take(5)
print
print show_channel_file.take(5)

[u'Hourly_Sports,21', u'Surreal_Sports,42', u'Hot_Games,85', u'Loud_Show,172', u'Baked_Games,347']

[u'Hourly_Sports,DEF', u'Hot_Talking,CNO', u'Almost_Cooking,MAN', u'Dumb_Show,BOB', u'PostModern_Sports,NOX']


In [2]:
# parse each dataset
def splitShowViews(line) :
    
    token = line.split(",")
    return ( token[0], int(token[1]) )

def splitShowChannel(line) :
    token = line.split(",")
    return ( token[0], token[1] )

show_views   = show_views_file.map( splitShowViews )
show_channel = show_channel_file.map( splitShowChannel )

# join the two datset
joined_dataset = show_views.join( show_channel )
print joined_dataset.take(5)

[(u'PostModern_Talking', (443, u'BOB')), (u'PostModern_Talking', (443, u'NOX')), (u'PostModern_Talking', (443, u'CNO')), (u'PostModern_Talking', (443, u'MAN')), (u'PostModern_Talking', (443, u'NOX'))]


In [3]:
channel_views = ( joined_dataset
                  .map( lambda s : ( s[1][1], s[1][0] ) ) # channel as key, count as value regardless of shows
                  .reduceByKey( lambda a, b : a + b ) # group by channels
                  .filter( lambda s : s[0] == "BAT" )) # extract the "BAT"
print channel_views.collect()

[(u'BAT', 5099141)]
